In [ ]:
# ============================================================
# 🏭 Task 1: Manufacturing Chatbot with Memory 
# ============================================================

In [ ]:
# Step 1: Install Dependencies
!pip install langchain langchain-groq gradio

In [2]:

# Step 2: Import Libraries
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
import gradio as gr
import os

C:\Users\cocsa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

# Step 3: Set your Groq API Key
# 👉 Replace with your actual Groq API key
os.environ["GROQ_API_KEY"] = "your_groq_api_key_here"

In [5]:
# Step 4: Initialize LLM (Groq)
llm = ChatGroq(
    api_key=os.environ["GROQ_API_KEY"],
    model="llama-3.1-8b-instant"  # Fast and capable conversational model
)

In [27]:
# Step 5: Add Memory to Chat
memory = ConversationBufferMemory(memory_key="history", return_messages=True)
prompt = PromptTemplate(
    input_variables=["history", "input"],
    template=(
        "You are a helpful manufacturing assistant.\n"
        "You assist operators, supervisors, and suppliers with production, maintenance, and inventory details.\n"
        "Use the conversation history below to maintain context.\n\n"
        "Conversation:\n{history}\n"
        "User: {input}\n"
        "Assistant:"
    ),
)

In [28]:

# Step 6: Create conversation chain
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True
)

In [29]:
# Step 7: Initialize JSON log file
json_file = "sample_conversations.json"

def init_json_log():
    if not os.path.exists(json_file):
        with open(json_file, "w") as f:
            json.dump([], f, indent=4)

def save_to_json(user, bot_response):
    with open(json_file, "r") as f:
        data = json.load(f)

    # Create new chat entry
    chat_entry = {
        "sender": "User",
        "message": user,
        "timestamp": str(datetime.utcnow())
    }
    bot_entry = {
        "sender": "Assistant",
        "message": bot_response,
        "timestamp": str(datetime.utcnow())
    }

    # Append entries
    if data and isinstance(data, list):
        data.append(chat_entry)
        data.append(bot_entry)
    else:
        data = [chat_entry, bot_entry]

    # Save updates
    with open(json_file, "w") as f:
        json.dump(data, f, indent=4)

# Initialize log file
init_json_log()

In [30]:
# Step 8: Define chatbot function
def chat_with_bot(message, history):
    try:
        response = conversation.predict(input=message)
        save_to_json(message, response)  # save each exchange
        return response
    except Exception as e:
        return f"⚠️ Error: {str(e)}"

In [ ]:
# Step 9: Launch Gradio chat interface
gr.ChatInterface(
    fn=chat_with_bot,
    title="🏭 Manufacturing Chatbot with Memory",
    description="Ask about production, maintenance, or operations. I remember previous queries!",
    theme="soft"
).launch()

C:\Users\cocsa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gradio\chat_interface.py:348: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.




> Entering new ConversationChain chain...
Prompt after formatting:
You are a helpful manufacturing assistant.
You assist operators, supervisors, and suppliers with production, maintenance, and inventory details.
Use the conversation history below to maintain context.

Conversation:
[]
User: hi whats up 
Assistant:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
You are a helpful manufacturing assistant.
You assist operators, supervisors, and suppliers with production, maintenance, and inventory details.
Use the conversation history below to maintain context.

Conversation:
[HumanMessage(content='hi whats up '), AIMessage(content='Not much, just getting ready for the morning shift. What about you? Do you need any assistance with production or inventory?')]
User: so how you doing today
Assistant:

> Finished chain.
